In [1]:
import cvxpy as cp

x = cp.Variable()

prob = cp.Problem(cp.Minimize(cp.maximum(0, x)), [])

prob.solve()

0.0

In [25]:
import cvxpy as cp
import numpy as np

# define dimensions
n, m = 100, 10

# define variables
w = cp.Variable(n, name='w')
w_change = cp.Variable(n, name='w_change')
f = cp.Variable(m, name='f')

# define parameters
a = cp.Parameter(n, name='a')
Ft = cp.Parameter((n, m), name='Ft')
Dt = cp.Parameter((n, n), diag=True, name='Dt')
k_tc = cp.Parameter(n, nonneg=True, name='k_tc')
k_sh = cp.Parameter(n, nonneg=True, name='k_sh')
w_prev = cp.Parameter(n, name='w_prev')
L = cp.Parameter(nonneg=True, name='L')

# define objective
objective = cp.Maximize(a@w
                        -cp.sum_squares(f)
                        -cp.sum_squares(Dt@w)
                        -k_tc@cp.abs(w_change)
                        +k_sh@cp.minimum(0, w))

# define constraints
constraints = [f == Ft.T@w, np.ones(n)@w == 1, 
               cp.norm(w, 1) <= L, w_change == w-w_prev]

# define problem
problem = cp.Problem(objective, constraints)

np.random.seed(0)
gamma = 10
alpha = 1000*(-1+2*np.random.rand(n))
kappa_tc = 10*np.ones(n)
kappa_sh = 20*np.ones(n)

a.value = alpha/gamma
Ft.value = np.random.randn(n, m)
Dt.value = np.diag(np.ones(n))
k_tc.value = kappa_tc/gamma
k_sh.value = kappa_sh/gamma
w_prev.value = np.zeros(n)
L.value = 1.6

val = problem.solve()

In [26]:
problem.is_dcp(dpp=True)

True

In [16]:
import cvxpy as cp

x = cp.Variable(3)
P = cp.Parameter((3,3))

prob = cp.Problem(cp.Minimize(cp.sum_squares(P@x)), [])

prob.is_dcp(dpp=True)

True

In [29]:
import cvxpy as cp

x = cp.Variable(3)
a = cp.Parameter(3)
b = cp.Parameter(3, nonneg=True)
c = cp.Parameter()

prob = cp.Problem(cp.Minimize(b@cp.abs(x-a)), [])

prob.is_dcp(dpp=True)

False